In [1]:
import numpy as np
import pandas as pd
import jieba

In [2]:
#read weekly_popular.xlsx
most_recent = pd.read_csv('most_recent.csv')
#drop nan
most_recent = most_recent.dropna()

In [3]:
def cut_words(column_name):
    column_cut = []
    #get column
    column = most_recent[column_name]
    for i in column:
        i = str(i)
        column_cut.append(jieba.lcut(i))
    wordlist = []
    for i in column_cut:
        for j in i:
            wordlist.append(j)
    #save word_list to word_list.csv
    wordlist = pd.DataFrame(wordlist)
    #drop nan
    wordlist = wordlist.dropna()
    wordlist.to_csv('{}_wordlist.csv'.format(column_name),index=False)
    #remove stop words from cn_stopwords.txt
    stopwords = [line.strip() for line in open('cn_stopwords.txt',encoding='UTF-8').readlines()]
    wordlist_removed = wordlist[~wordlist.isin(stopwords)]
    #drop nan
    wordlist_removed = wordlist_removed.dropna()
    #save word_list to word_list_noStopWords.csv
    wordlist_removed.to_csv('{}_wordlist_noStopWords.csv'.format(column_name),index=False)

In [4]:
cut_words('title')
cut_words('desc')
cut_words('dynamic')

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/ts/yc6c7tl96hjdf33yyy_9y96m0000gn/T/jieba.cache
Loading model cost 0.853 seconds.
Prefix dict has been built successfully.


In [5]:
def perc_missing(df):
    '''prints out columns with missing values with its %'''
    for col in df.columns:
        pct = df[col].isna().mean() * 100
        if (pct != 0):
            print('{} => {}%'.format(col, round(pct, 2)))

In [6]:
perc_missing(most_recent)

In [7]:
most_recent = pd.read_csv('most_recent.csv')

most_recent = most_recent.drop([11603,13849], axis=0)
most_recent = most_recent.drop('dimension',axis=1)
#rearange the index
most_recent = most_recent.reset_index(drop=True)

#make each row of stat do dict
stat = most_recent['stat']
stat_dict = []
for i in stat:
    #if i is nan, make it to 0
    if type(i) == float:
        i = "{'aid': 0, 'view': 1, 'danmaku': 0, 'reply': 0, 'favorite': 0, 'coin': 0, 'share': 0, 'now_rank': 0, 'his_rank': 0, 'like': 0, 'dislike': 0, 'evaluation': 0, 'argue_msg': 0}"
    sd = eval(i)
    stat_dict.append(sd)
#make stat_dict to dataframe of dict
stat_dict = pd.DataFrame(stat_dict)

#drop columns that are not needed
stat_dict = stat_dict.drop(['aid','now_rank','his_rank','evaluation','argue_msg', 'dislike'],axis=1)
#replace stat with stat_dict
most_recent = most_recent.drop('stat',axis=1)
#concat most_recent and stat_dict and based on row number
most_recent = pd.concat([most_recent,stat_dict],axis=1)

In [8]:
owner = most_recent['owner']
owner_dict = []
for i in owner:
    #if i is nan, make it to 0
    if type(i) == float:
        i = "{'mid' : 0, 'name' : 0, 'face' : 0}"
    od = eval(i)
    owner_dict.append(od)
#make stat_dict to dataframe of dict
owner_dict = pd.DataFrame(owner_dict)

#drop columns that are not needed
owner_dict = owner_dict.drop('face',axis=1)
#replace stat with stat_dict
most_recent = most_recent.drop('owner',axis=1)
#concat most_recent and stat_dict and based on row number
most_recent = pd.concat([most_recent,owner_dict],axis=1)

In [9]:
#fill nan with 0
most_recent = most_recent.fillna(0)
coin = most_recent['coin']
favorite = most_recent['favorite']
danmaku = most_recent['danmaku']
reply = most_recent['reply']
view = most_recent['view']
like = most_recent['like']
share = most_recent['share']
for i in range(len(view)):
    if view[i] == 0:
        view[i] = 1
recommandation_rate = (coin * 1.2 + favorite * 0.9 + danmaku * 1.2 + reply * 1.2 + view * 0.75 + like * 1.2 + share * 1.8) / view
most_recent['recommandation_rate'] = recommandation_rate

/var/folders/ts/yc6c7tl96hjdf33yyy_9y96m0000gn/T/ipykernel_21623/3508209395.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  view[i] = 1


In [10]:
#compute the recommandation rate mean and variance
#find outlyer in recommandation_rate
rr = most_recent['recommandation_rate']
rr_mean = rr.mean()
rr_std = rr.std()
print(rr_mean)
print(rr_std)

0.8782552203877642
0.19304744638383986


In [11]:
#add a column of recommandation_rate_level
rr_level = []
for i in rr:
    if i > rr_mean + 2 * rr_std:
        rr_level.append('highly recommend')
    elif i > 1:
        rr_level.append('recommend')
    elif i > rr_mean:
        rr_level.append('neutral')
    else:
        rr_level.append('not recommend')
most_recent['recommandation_rate_level'] = rr_level

In [12]:
most_recent.tail()

,Unnamed: 0,tid,tname,title,desc,duration,dynamic,bvid,view,danmaku,reply,favorite,coin,share,like,mid,name,recommandation_rate,recommandation_rate_level
14995,0.0,231,计算机技术,【Unity3D VFX】Unity3D VFX 骨骼绑定的闪电特效,0,69,#Unity3D##VFX##骨骼绑定#,BV1jE411A7xC,1178,0,0,4,0,0,4,435663342,技术侠,0.757131,not recommend
14996,0.0,157,美妆护肤,#GRWM 一个不正经的蜜桃妆,新人up来报道啦！\n视频可能有些地方不足比如说我的素颜啦 还请你们多多包涵 如果你们有建议...,525,#日常# #化妆# #GRWM#,BV1jE411A7xy,12,0,0,0,0,0,1,160192438,aaaa000_,0.850000,not recommend
14997,0.0,174,其他,PN-CT300F边压强度试验,0,115,#边压##知识分享官#,BV1jE411A7x1,183,0,0,1,0,1,0,522203960,杭州品享科技,0.764754,not recommend
14998,0.0,137,明星综合,王嘉尔《数嘉宝贝》之100种变身方式ಥ_ಥ,-,35,#明星# #王嘉尔# #王嘉尔JACKSON#《100ways》新歌来袭！古装我可！,BV1jE411A7x2,29,0,0,0,0,0,1,38389510,Far一,0.791379,not recommend
14999,0.0,172,手机游戏,猫和老鼠：小朋友你是否有很多问号？？？,猫和老鼠：如何制裁史上最脏套路？选海盗就对了！完全不起作用,103,#猫和老鼠##手游##游戏视频#,BV1jE411A7xz,2792,3,12,7,2,2,180,6986914,黑猫游戏鸡,0.838216,not recommend


In [13]:
#save to csv
most_recent.to_csv('most_recent_updated.csv',index=False)